<a href="https://colab.research.google.com/github/AlexKressner/OR_Workshop/blob/main/Ganzzahlige_Optimierung_Spediteur_erweitert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auswahlproblem Spediteur, Erweiterung

In [16]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden der Packages
import pandas as pd
from ortools.linear_solver import pywraplp
from google.colab import drive

In [17]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Daten aus Google Drive laden


1.   Problemrelevante Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden

In [18]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
path = "/content/drive/MyDrive/OR_Workshop/Daten"

In [20]:
orders = pd.read_csv(f"{path}/Spediteur_Auftraege.csv", sep=";", decimal=",")
orders

,Auftrag,Deckungsbeitrag,Volumen
0,A1,10,3.00
1,A2,15,4.50
2,A3,25,9.25
3,A4,5,1.20
4,A5,11,4.00
5,A6,8,2.70


In [21]:
lkw = pd.read_csv(f"{path}/Spediteur_LKW.csv", sep=";", decimal=",")
lkw

,LKW,Ladevolumen,Fixkosten
0,L1,10,12
1,L2,8,7
2,L3,5,5


## Indexmengen

In [22]:
I = orders["Auftrag"].unique().tolist() # Menge der Aufträge

In [23]:
J = lkw["LKW"].unique().tolist() # Menge der LKW

## Entscheidungsvariablen

In [24]:
# Definition der Entscheidungsvariablen
x={}
y={}
for i in I:
  for j in J: 
    x[i,j] = solver.BoolVar(f"{i,j}")

for j in J:
    y[j] = solver.BoolVar(f"{j}")
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 21


## Parameter

In [25]:
# Deckungsbeiträge
c = orders.set_index(["Auftrag"]).to_dict("dict")["Deckungsbeitrag"]
c

{'A1': 10, 'A2': 15, 'A3': 25, 'A4': 5, 'A5': 11, 'A6': 8}

In [26]:
# Auftragsvolumen
a = orders.set_index(["Auftrag"]).to_dict("dict")["Volumen"]
a

{'A1': 3.0, 'A2': 4.5, 'A3': 9.25, 'A4': 1.2, 'A5': 4.0, 'A6': 2.7}

In [27]:
# Fixkosten der LKW
f = lkw.set_index(["LKW"]).to_dict("dict")["Fixkosten"]
f

{'L1': 12, 'L2': 7, 'L3': 5}

In [28]:
# Kapazität der LKW
b = lkw.set_index(["LKW"]).to_dict("dict")["Ladevolumen"]
b

{'L1': 10, 'L2': 8, 'L3': 5}

## Zielfunktion

In [29]:
# Maximierung der gesamten Deckungsbeiträge
solver.Maximize(sum(c[i]*x[i,j] for i in I for j in J) - sum(f[j] * y[j] for j in J))

## Nebenbedingungen

In [30]:
# LKW Kapazitäten
for j in J:
  solver.Add(sum(a[i]*x[i,j] for i in I)<=b[j] * y[j])

# Einmalige Verladung
for i in I:
  solver.Add(sum(x[i,j] for j in J)<=1)

## Berechnung Lösung

In [31]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', solver.Objective().Value())
    for j in J:
      for i in I:
        if x[i,j].solution_value()>0:
          print(f'{i,j} =', round(x[i,j].solution_value()))
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 40.0
('A3', 'L1') = 1
('A4', 'L2') = 1
('A5', 'L2') = 1
('A6', 'L2') = 1
('A2', 'L3') = 1
